In [ ]:
from cleanfid import fid
import os
import PIL.Image
import torch
from torch import nn
import numpy as np

In [ ]:
loss = nn.MSELoss(reduction='mean')
prefix = 'Dawn'

def compute_mse(dataset, fdir1, fdir2):
    
    mse_list = []

    
    for file in os.listdir(fdir1):
        if file.endswith('.png'):
            file1 = np.array(PIL.Image.open(os.path.join(fdir1, file))) /127.5 -1
            if dataset != prefix:
                file2 = np.array(PIL.Image.open(os.path.join(fdir2, file.replace("_" + dataset.replace(prefix+'_', ""), ""))))/127.5 -1
            else:
                file2 = np.array(PIL.Image.open(os.path.join(fdir2, file)))/127.5 -1
            mse = np.mean(np.square(file1 - file2))
            mse_list.append(mse)
            # print(mse)
#             if file1.ndim == 2:
#                 file1 = file1[:, :, np.newaxis] # HW => HWC            
#             if file2.ndim == 2:
#                 file2 = file2[:, :, np.newaxis] # HW => HWC
                
#             file1 = torch.tensor(file1.transpose(2, 0, 1)).to(torch.float32) / 127.5 - 1 # HWC => CHW
#             file2 = torch.tensor(file2.transpose(2, 0, 1)).to(torch.float32) / 127.5 - 1 # HWC => CHW
            
#             output = loss(file1, file2)
#             print(output)
    return np.mean(mse_list)

In [ ]:
pre_path = '/home/featurize/data'
# pre_path = '/home/featurize/data/LOL'
datasets = os.listdir(pre_path)
# prefix = 'trainB'
datasets = [i for i in datasets if i.startswith(prefix + '_')]
datasets, len(datasets)

In [ ]:
fids = []
mses = []
fdir_ = '/home/featurize/data/' + prefix
# fdir_ = '/home/featurize/data/LOL/trainB'


for dataset in datasets:
    custom_name = dataset.replace(prefix + "_", "").lower()
    dataset_path = os.path.join(pre_path, dataset)
    # fid.make_custom_stats(custom_name, dataset_path, mode="clean")
    # print("{} stat has been created !".format(custom_name))
    fid_ = fid.compute_fid(dataset_path, fdir_)
    fids.append(fid_)
    mse_ = compute_mse(dataset,dataset_path, fdir_)
    mses.append(mse_)
    print("The FID / MSE between folder1 {} and folder2 {} is {} / {}".format(custom_name, prefix, fid_, mse_))

    print("————"*30)


In [ ]:
fid_dict = {}
threshold = 0.02
txt = open(prefix + '.txt', 'w')

for i in range(len(datasets)):
    dataset = datasets[i]
    fid_ = fids[i]
    mse_ = mses[i]
    
    txt.write("{} ——————— FID: {:.3f}, MSE: {:.3f}\n".format(dataset, fid_, mse_))

    
    if mse_ > threshold:
        fid_dict[dataset] = fid_
        print("{} ——————— FID: {:.3f}, MSE: {:.3f}".format(dataset, fid_, mse_))
        
    if dataset.endswith('Flip') or dataset.endswith('Affine'):
        print("__")

txt.close()

In [ ]:
from pprint import pprint
pprint(sorted(fid_dict.items(), key = lambda kv:(kv[1], kv[0])))     

In [ ]:
aug_fid = sorted(fid_dict.items(), key = (lambda kv : (kv[1], kv[0])))
num = 6
aug_chosen = [kv[0] for kv in aug_fid[:num]]
print("The augmentation chosen by MineAug is  {}".format(aug_chosen))